In [3]:
from tutorial_utils import magma_to_verilog_string, smt_to_smtlib_string

import hwtypes as hw
import magma as m

from peak import Peak # the base class of Peak circuits
from peak import family_closure
from peak import family


In the previous section we introduced hwtypes and how we can use python to meta-program it with higher order functions.
In this section we will introduce peak which extends the expression language of hwtypes. 

Peak circuits are defined in a python class.  Similar to hwtypes, a peak program can be executed in pure python, symbolically executed with smt, or generate a circuit with magma.  Each of these interpretations must additionally supply implementations of primitives such as registers which are not defined in hwtypes.   A Peak class is defined in a "family closure" which provides access to the type constructors and primitives of the interpretation. 

Peak circuits define their behavior in there `__call__` method.  Peak aims to match the semantics of a normal, to facilitate this, Peak allows the use of `if` statement on live values.  This achieved by compiling them to `ite`s and hence they are subject to same constraints as those described in the previous section.  Subcomponent of a circuit are defined in its `__init__` method.  

In the following example we will demonstrate a basic peak program.

By default the `family_closure` decorator associates with the base families (see: `peak.family`) which have constructors for bit types, bitvector types, registers, and ADTs.  However, these can be extend with arbitrary modules and types as needed. An extend set of families can be passed to the family by passing an object (typically a module) with attributes `PyFamily`, `SMTFamily`, and `MagmaFamily` to the `family_closure` decorator. E.g. `@family_closure(peak.family)` would explicity bind the closure to the defauly families. 

This association allows the family closure to provide a convenient syntax shortcut for calling the closure:

```
closure.Py == closure(family.PyFamily())
closure.SMT == clolsure(family.SMTFamily())
closure.Magma == closure(family.MagmaFamily())
```

`family.assemble` evokes the peak compiler passing the current namespace to the compiler with `locals(), globals()`. 

The peak compiler requires type annotation on `__call__` method in order generate ports in a magma context.

add motivation

In [ ]:
PyDataT = hw.BitVector[8]
SmtDataT = hw.SMTBitVector[8]
MagmaDataT = m.Bits[8]

@family_closure
def closure(family):
    DataT = family.BitVector[8]
    Bit = family.Bit
    @family.assemble(locals(), globals())
    class ALU(Peak):
        def __call__(self, op: Bit, in_0: DataT, in_1: DataT) -> DataT:
            if op:
                return in_0 + in_1
            else:
                return in_0 - in_1
    return ALU

The peak compiler transorms this to the following hwtypes program (equivelent up to a renaming):


In [ ]:
# hwtypes impl
@family_closure
def closure(family):
    DataT = family.BitVector[8]
    Bit = family.Bit
    class ALU(Peak):
        def __call__(self, self, op: Bit, in_0: DataT, in_1: DataT) -> DataT:
            _cond_0 = op
            _return_0 = in_0 + in_1
            _return_1 = in_0 + in_1
            return Bit.ite(_cond_0, _return_0, return_1)


In [ ]:
AluTpy = closure.Py
alu = AluTpy()
print(alu(hw.Bit(0), PyDataT(2), PyDataT(1)))

Note that alu cannot be invoked with magma or smt types as the `if` will not be compiled

In [ ]:
print(alu(hw.SMTBit(name='op'), SmtDataT(name='x'), SmtDataT(name='y')))

However if we invoke it with the proper family it works as expected

In [ ]:
AluTsmt = closure.SMT
alu = AluTsmt()
results = alu(hw.SMTBit(name='op'), hw.SMTBitVector[8](name='x'), hw.SMTBitVector[8](name='y'))
print(smt_to_smtlib_string(results))

The magma family works slightly differently not only does it rewrite the `__call__` method it further reinterprets the entire class as a circuit.  It returns a magma circuit definition, which does not need to wrapped (compare with the hwtypes programs shown in the previous section).  

In [ ]:
AluTmagma = closure.Magma
print(magma_to_verilog_string(AluTmagma))

As mentioned in the intro, subcomponent can be declared in the `__init__` method.  Below we wrap the original ALU to extend it with additional instructions.

In [ ]:
@family_closure
def closure2(family):
    BV = family.BitVector
    DataT = BV[8]
    Bit = family.Bit
    AluT = closure(family)
    @family.assemble(locals(), globals())
    class ALU(Peak):
        def __init__(self):
                self.alu = AluT()
            
        def __call__(self, op: BV[2], in_0: DataT, in_1: DataT) -> DataT:
            if ~op[1]: # 01 and 10
                return self.alu(op[0],in_0, in_1)
            elif op[0]:
                return in_0 & in_1
            else: # 00
                return in_0 | in_1
    return ALU

something something ADTs better than raw bitvectors

In [ ]:
class ISA(hw.Enum):
    Add = hw.new_instruction()
    Sub = hw.new_instruction()
    And = hw.new_instruction()
    Or = hw.new_instruction()

@family_closure
def closure3(family):
    BV = family.BitVector
    DataT = BV[8]
    Bit = family.Bit
    AluT = closure(family)
    @family.assemble(locals(), globals())
    class ALU(Peak):            
        def __call__(self, op: ISA, in_0: DataT, in_1: DataT) -> DataT:
            if op == ISA.Add:
                return in_0 + in_1
            elif op == ISA.Sub:
                return in_0 - in_1
            elif op == ISA.And: 
                return in_0 & in_1
            else: 
                return in_0 | in_1
    return ALU

In [8]:
class Arith(hw.Enum):
    Add = hw.new_instruction()
    Sub = hw.new_instruction()
    
class Bitwise(hw.Enum):
    And = hw.new_instruction()
    Or = hw.new_instruction()
    
ISA = hw.Sum[Arith, Bitwise]
@family_closure
def lu_fc(family):
    BV = family.BitVector
    DataT = BV[8]

    @family.assemble(locals(), globals())
    class LU(Peak):
        def __call__(self, op: Bitwise, in_0: DataT, in_1: DataT) -> DataT:
            if op == Bitwise.And:
                return in_0 & in_1
            else:
                return in_0 | in_1
    return LU


@family_closure
def au_fc(family):
    BV = family.BitVector
    DataT = BV[8]

    @family.assemble(locals(), globals())
    class AU(Peak):
        def __call__(self, op: Arith, in_0: DataT, in_1: DataT) -> DataT:
            if op == Arith.Add:
                return in_0 + in_1
            else:
                return in_0 - in_1
    return AU


@family_closure
def alu_fc(family):
    BV = family.BitVector
    DataT = BV[8]
    LU_t = lu_fc(family)
    AU_t = au_fc(family)
    
    
    @family.assemble(locals(), globals())
    class ALU(Peak):           
        def __init__(self):
            self.au = AU_t()
            self.lu = LU_t()
            
        def __call__(self, op: ISA, in_0: DataT, in_1: DataT) -> DataT:
            if op[Arith].match:
                return self.au(op[Arith].value, in_0, in_1)
            else:
                return self.lu(op[Bitwise].value, in_0, in_1)
    return ALU